# NBA MCP Data Exploration

This notebook demonstrates how to explore the NBA database using the MCP server.

## Prerequisites
- MCP server running (`./scripts/start_mcp_server.sh`)
- Environment variables configured (`.env` file)
- Required Python packages installed

In [ ]:
# Setup and imports
import sys
import os
import asyncio
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent
sys.path.insert(0, str(project_root))

# Load environment variables
from dotenv import load_dotenv
load_dotenv(project_root / '.env')

# Import MCP client
from synthesis.mcp_client import MCPClient

print("✓ Environment loaded")
print(f"✓ Project root: {project_root}")

In [ ]:
# Connect to MCP server
mcp_client = MCPClient()
connected = await mcp_client.connect()

if connected:
    print("✅ Connected to MCP server")
else:
    print("❌ Failed to connect to MCP server")
    print("Make sure the server is running: ./scripts/start_mcp_server.sh")

## 1. Explore Available Tables

In [ ]:
# List all available tables
tables = await mcp_client.call_tool("list_tables", {})

if tables.get("success"):
    table_list = tables.get("tables", [])
    print(f"Found {len(table_list)} tables:\n")
    for table in table_list:
        print(f"  • {table}")
else:
    print(f"Error: {tables.get('error')}")

## 2. Examine Table Schemas

In [ ]:
# Get schema for a specific table
table_name = "games"  # Change this to explore different tables

schema = await mcp_client.call_tool("get_table_schema", {"table_name": table_name})

if schema.get("success"):
    print(f"Schema for {table_name}:\n")
    
    # Display as DataFrame for better formatting
    columns_df = pd.DataFrame(schema.get("columns", []))
    display(columns_df)
else:
    print(f"Error: {schema.get('error')}")

## 3. Query Data

In [ ]:
# Example: Get recent games
query = """
SELECT 
    game_id,
    game_date,
    home_team_id,
    away_team_id,
    home_team_score,
    away_team_score
FROM games
ORDER BY game_date DESC
LIMIT 10
"""

result = await mcp_client.call_tool("query_database", {"sql": query})

if result.get("success"):
    # Convert to DataFrame
    df = pd.DataFrame(result.get("results", []))
    print(f"Recent games ({len(df)} rows):\n")
    display(df)
else:
    print(f"Error: {result.get('error')}")

## 4. Statistical Analysis

In [ ]:
# Example: Analyze scoring trends
query = """
SELECT 
    EXTRACT(YEAR FROM game_date) as season,
    AVG(home_team_score + away_team_score) as avg_total_points,
    COUNT(*) as games_count
FROM games
WHERE game_date >= '2015-01-01'
GROUP BY EXTRACT(YEAR FROM game_date)
ORDER BY season
"""

result = await mcp_client.call_tool("query_database", {"sql": query})

if result.get("success"):
    df = pd.DataFrame(result.get("results", []))
    
    # Plot scoring trends
    plt.figure(figsize=(12, 6))
    plt.plot(df['season'], df['avg_total_points'], marker='o', linewidth=2)
    plt.title('Average Total Points per Game by Season', fontsize=14)
    plt.xlabel('Season')
    plt.ylabel('Average Total Points')
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    display(df)
else:
    print(f"Error: {result.get('error')}")

## 5. Access S3 Data

In [ ]:
# List files in S3
s3_files = await mcp_client.call_tool("list_s3_files", {"prefix": "", "max_keys": 10})

if s3_files.get("success"):
    files = s3_files.get("files", [])
    print(f"Sample S3 files ({len(files)} shown):\n")
    for f in files:
        print(f"  • {f['key']} ({f.get('size', 0) / 1024:.1f} KB)")
else:
    print(f"Error: {s3_files.get('error')}")

## 6. Cleanup

In [ ]:
# Disconnect from MCP server
await mcp_client.disconnect()
print("✓ Disconnected from MCP server")